In [1]:
import pandas as pd
df = pd.read_csv('mushroom_dataset.csv')
print(len(df[df['class']=='p']))
df

3916


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [2]:
def encode_onehot(_df, _f):
    _df2 = pd.get_dummies(_df[_f], prefix='', prefix_sep='').groupby(level=0, axis=1).max().add_prefix(_f+' - ')
    _df3 = pd.concat([_df, _df2], axis=1)
    _df3 = _df3.drop([_f], axis=1)
    return _df3

In [3]:
names = list(df)
for name in names:#automating binary replacements
    if df[name].nunique()==2:
        df[name].replace([df[name].unique()[0],df[name].unique()[1]],[0,1],inplace=True)
    else:
        df = encode_onehot(df,name)
df

,class,bruises,gill-attachment,gill-spacing,gill-size,stalk-shape,cap-shape - b,cap-shape - c,cap-shape - f,cap-shape - k,...,population - s,population - v,population - y,habitat - d,habitat - g,habitat - l,habitat - m,habitat - p,habitat - u,habitat - w
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,1,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
8120,1,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
8121,1,1,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8122,0,1,0,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0


In [4]:
#y = df['class']
#X = df.drop('class',axis=1)

dfX = df.loc[:, df.columns != 'class']
dfy = df.loc[:, df.columns == 'class'].values.ravel()

# Sanity check
print(f'N={len(dfX)}, M={len(dfX.columns)}')
X = dfX.values
y = dfy

N=8124, M=112


In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# 10-fold CV evaluation of a classifier
def eval_classifier(_clf, _X, _y):
    accuracies = []
    kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
    for train_index, test_index in kf.split(_X, _y):
        _clf.fit(_X[train_index], _y[train_index])
        y_pred = _clf.predict(_X[test_index])
        accuracies += [accuracy_score(_y[test_index], y_pred)]
    return np.array(accuracies)




gnb = GaussianNB()

In [6]:
acc = eval_classifier(GaussianNB(), X, y)
print(f'Naive Bayes CV accuracy={np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Naive Bayes CV accuracy=0.91 ±0.084


In [7]:
from sklearn.svm import SVC
svc = SVC(kernel='linear', probability=True)
acc = eval_classifier(svc, X, y)
print(f'SVC accuracy={np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

SVC accuracy=0.97 ±0.070


In [8]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()

acc = eval_classifier(mlp, X, y)
print(f'MLP accuracy={np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

MLP accuracy=0.96 ±0.094


In [9]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

acc = eval_classifier(dtc, X, y)
print(f'Decision tree accuracy={np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Decision tree accuracy=0.96 ±0.097


In [10]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

acc = eval_classifier(rfc, X, y)
print(f'Random Forest accuracy={np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Random Forest accuracy=0.97 ±0.094


In [11]:
# Let's build a single weak learner using GaussianNB by subset of features
def weakNB_fit(_list_cols, _X, _y):
    Xs = _X[:,_list_cols]
    return GaussianNB(priors=NBpriors).fit(Xs, _y)  # return a single NaiveBayes

def weakNB_predict(_clf, _list_cols, _X):
    Xs = _X[:,_list_cols]
    return _clf.predict(Xs), _clf.predict_proba(Xs)

# Use _m features randomly selected from _M features
# a total of n_estimators many weak learners
def features_randomsubset(_M, _m, n_estimators=1):
    from numpy.random import choice
    # returns a list of list of column choices - subset features
    return [choice(_M, _m, replace=False) for _ in range(n_estimators)]

In [12]:
def eval_singleweak(_X, _y, _niters, _nfeatures):
    accuracies = []
    for _ in range(_niters):
        # Keep the subset features (i.e. columns) the same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, n_estimators=1)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            clf = weakNB_fit(cols[0], _X[train_index], _y[train_index])
            y_pred, y_prob = weakNB_predict(clf, cols[0], _X[test_index])
            accuracies += [accuracy_score(_y[test_index], y_pred)]
    return np.array(accuracies)

In [13]:
def ensembleNB_fit(_ensemble_cols, _X, _y):
    # the list of ensemble columns have a column list for every member of the ensemble
    n_estimators = len(_ensemble_cols)
    # list of weak learners
    ensemble_clf = []
    for j in range(n_estimators):
        ensemble_clf += [weakNB_fit(_ensemble_cols[j], _X, _y)]

    return ensemble_clf

# Using trained ensemble, predict the outcome by majority voting
def ensembleNB_predict(_ensemble_clf, _ensemble_cols, _Xtest):
    from collections import defaultdict
    n_estimators = len(_ensemble_clf)
    assert n_estimators==len(_ensemble_cols)  # Error check
    # weak learner predictions
    ypred_e, yprob_e = [], []
    for j in range(n_estimators):
        res = weakNB_predict(_ensemble_clf[j], _ensemble_cols[j], _Xtest)
        ypred_e += [res[0]]
        yprob_e += [res[1]]  # score/probability of the prediction
    # majority voting for each data point in _Xtest
    ypred = []
    for i in range(_Xtest.shape[0]):
        ypred_scores = defaultdict(float)
        for j in range(n_estimators):
            for c, p in enumerate(yprob_e[j][i]):
                # a proper score is necessary
                ypred_scores[c] += p
        ix = max(ypred_scores.items(), key=lambda a: a[1])
        ypred += [ix[0]]

    return np.array(ypred)

In [14]:
def eval_ensemble(_X, _y, _niter, _n_estimators, _nfeatures):
    accuracies = []
    single_accuracy = []
    for _ in range(_niter):
        # Keep subset features, columns same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, n_estimators=_n_estimators)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            e_clf = ensembleNB_fit(cols, _X[train_index], _y[train_index])
            y_pred = ensembleNB_predict(e_clf, cols, _X[test_index])
            single_y_pred = e_clf[0].fit(_X[train_index], _y[train_index]).predict(_X[test_index])
            accuracies += [accuracy_score(_y[test_index], y_pred)]
            single_accuracy += [accuracy_score(_y[test_index], single_y_pred)]
    return np.array(accuracies),np.array(single_accuracy)

In [15]:
counts = np.unique(y, return_counts=True)
NBpriors = [counts[1][0]/len(y), counts[1][1]/len(y)]
acc = eval_ensemble(X, y, 5, 100, 7)#5 iterations this takes forever
print(f'Ensemble learners average Acc= {np.mean(acc[0]):.2f} {chr(177)}{np.std(acc[0]):.3f}')
print(f'First weak learners average Acc= {np.mean(acc[1]):.2f} {chr(177)}{np.std(acc[1]):.3f}')

Ensemble learners average Acc= 0.86 ±0.113
First weak learners average Acc= 0.91 ±0.084


In [19]:
# Let's build a single weak learner using Neural Network by subset of features
def weakNN_fit(_list_cols, _X, _y):
    Xs = _X[:,_list_cols]
    return MLPClassifier(hidden_layer_sizes=(5, 2),max_iter=30,tol = 1e-1).fit(Xs, _y)  # return a single NN

def weakNN_predict(_clf, _list_cols, _X):
    Xs = _X[:,_list_cols]
    return _clf.predict(Xs), _clf.predict_proba(Xs)

# Use _m features randomly selected from _M features
# a total of n_estimators many weak learners
def features_randomsubset(_M, _m, n_estimators=1):
    from numpy.random import choice
    # returns a list of list of column choices - subset features
    return [choice(_M, _m, replace=False) for _ in range(n_estimators)]

def eval_singleweak(_X, _y, _niters, _nfeatures):
    accuracies = []
    for _ in range(_niters):
        # Keep the subset features (i.e. columns) the same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, n_estimators=1)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            clf = weakNN_fit(cols[0], _X[train_index], _y[train_index])
            y_pred, y_prob = weakNN_predict(clf, cols[0], _X[test_index])
            accuracies += [accuracy_score(_y[test_index], y_pred)]
    return np.array(accuracies)

def ensembleNN_fit(_ensemble_cols, _X, _y):
    # the list of ensemble columns have a column list for every member of the ensemble
    n_estimators = len(_ensemble_cols)
    # list of weak learners
    ensemble_clf = []
    for j in range(n_estimators):
        ensemble_clf += [weakNN_fit(_ensemble_cols[j], _X, _y)]

    return ensemble_clf

# Using trained ensemble, predict the outcome by majority voting
def ensembleNN_predict(_ensemble_clf, _ensemble_cols, _Xtest):
    from collections import defaultdict
    n_estimators = len(_ensemble_clf)
    assert n_estimators==len(_ensemble_cols)  # Error check
    # weak learner predictions
    ypred_e, yprob_e = [], []
    for j in range(n_estimators):
        res = weakNN_predict(_ensemble_clf[j], _ensemble_cols[j], _Xtest)
        ypred_e += [res[0]]
        yprob_e += [res[1]]  # score/probability of the prediction
    # majority voting for each data point in _Xtest
    ypred = []
    for i in range(_Xtest.shape[0]):
        ypred_scores = defaultdict(float)
        for j in range(n_estimators):
            for c, p in enumerate(yprob_e[j][i]):
                # a proper score is necessary
                ypred_scores[c] += p
        ix = max(ypred_scores.items(), key=lambda a: a[1])
        ypred += [ix[0]]

    return np.array(ypred)

def eval_ensembleNN(_X, _y, _niter, _n_estimators, _nfeatures):
    accuracies = []
    single_accuracy = []
    for _ in range(_niter):
        # Keep subset features, columns same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, n_estimators=_n_estimators)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            e_clf = ensembleNN_fit(cols, _X[train_index], _y[train_index])
            y_pred = ensembleNN_predict(e_clf, cols, _X[test_index])
            single_y_pred = e_clf[0].fit(_X[train_index], _y[train_index]).predict(_X[test_index])
            accuracies += [accuracy_score(_y[test_index], y_pred)]
            single_accuracy += [accuracy_score(_y[test_index], single_y_pred)]
    return np.array(accuracies),np.array(single_accuracy)

counts = np.unique(y, return_counts=True)
acc = eval_ensembleNN(X, y, 2, 100, 3)#5 iterations this takes forever
print(f'Ensemble learners average Acc= {np.mean(acc[0]):.2f} {chr(177)}{np.std(acc[0]):.3f}')
print(f'First weak learners average Acc= {np.mean(acc[1]):.2f} {chr(177)}{np.std(acc[1]):.3f}')

Ensemble learners average Acc= 0.89 ±0.120
First weak learners average Acc= 0.75 ±0.235


In [20]:
# Let's build a single weak learner using DecisionTree by subset of features
def weakDT_fit(_list_cols, _X, _y):
    Xs = _X[:,_list_cols]
    return DecisionTreeClassifier(max_depth=5,max_features=5).fit(Xs, _y)  # return a single DT

def weakDT_predict(_clf, _list_cols, _X):
    Xs = _X[:,_list_cols]
    return _clf.predict(Xs), _clf.predict_proba(Xs)

# Use _m features randomly selected from _M features
# a total of n_estimators many weak learners
def features_randomsubset(_M, _m, n_estimators=1):
    from numpy.random import choice
    # returns a list of list of column choices - subset features
    return [choice(_M, _m, replace=False) for _ in range(n_estimators)]

def eval_singleweak(_X, _y, _niters, _nfeatures):
    accuracies = []
    for _ in range(_niters):
        # Keep the subset features (i.e. columns) the same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, n_estimators=1)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            clf = weakDT_fit(cols[0], _X[train_index], _y[train_index])
            y_pred, y_prob = weakDT_predict(clf, cols[0], _X[test_index])
            accuracies += [accuracy_score(_y[test_index], y_pred)]
    return np.array(accuracies)

def ensembleDT_fit(_ensemble_cols, _X, _y):
    # the list of ensemble columns have a column list for every member of the ensemble
    n_estimators = len(_ensemble_cols)
    # list of weak learners
    ensemble_clf = []
    for j in range(n_estimators):
        ensemble_clf += [weakDT_fit(_ensemble_cols[j], _X, _y)]

    return ensemble_clf

# Using trained ensemble, predict the outcome by majority voting
def ensembleDT_predict(_ensemble_clf, _ensemble_cols, _Xtest):
    from collections import defaultdict
    n_estimators = len(_ensemble_clf)
    assert n_estimators==len(_ensemble_cols)  # Error check
    # weak learner predictions
    ypred_e, yprob_e = [], []
    for j in range(n_estimators):
        res = weakDT_predict(_ensemble_clf[j], _ensemble_cols[j], _Xtest)
        ypred_e += [res[0]]
        yprob_e += [res[1]]  # score/probability of the prediction
    # majority voting for each data point in _Xtest
    ypred = []
    for i in range(_Xtest.shape[0]):
        ypred_scores = defaultdict(float)
        for j in range(n_estimators):
            for c, p in enumerate(yprob_e[j][i]):
                # a proper score is necessary
                ypred_scores[c] += p
        ix = max(ypred_scores.items(), key=lambda a: a[1])
        ypred += [ix[0]]

    return np.array(ypred)

def eval_ensembleDT(_X, _y, _niter, _n_estimators, _nfeatures):
    accuracies = []
    single_accuracy = []
    for _ in range(_niter):
        # Keep subset features, columns same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, n_estimators=_n_estimators)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            e_clf = ensembleDT_fit(cols, _X[train_index], _y[train_index])
            y_pred = ensembleDT_predict(e_clf, cols, _X[test_index])
            single_y_pred = e_clf[0].fit(_X[train_index], _y[train_index]).predict(_X[test_index])
            accuracies += [accuracy_score(_y[test_index], y_pred)]
            single_accuracy += [accuracy_score(_y[test_index], single_y_pred)]
    return np.array(accuracies),np.array(single_accuracy)

counts = np.unique(y, return_counts=True)
acc = eval_ensembleDT(X, y, 5, 100, 5)#5 iterations this takes forever
print(f'Ensemble learners average Acc= {np.mean(acc[0]):.2f} {chr(177)}{np.std(acc[0]):.3f}')
print(f'First weak learners average Acc= {np.mean(acc[1]):.2f} {chr(177)}{np.std(acc[1]):.3f}')

Ensemble learners average Acc= 0.91 ±0.115
First weak learners average Acc= 0.85 ±0.139


In [105]:
NUM_WEAK = 5
def eval_single_weak(_X, _y, _niters, _nfeatures,classifier):
    accuracies = []
    for _ in range(_niters):
        # Keep the subset features (i.e. columns) the same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, n_estimators=1)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            clf = classifier.fit(_X[train_index], _y[train_index])
            y_pred = classifier.predict(_X[test_index])
            accuracies += [accuracy_score(_y[test_index], y_pred)]
    return np.array(accuracies)

gaussian_ensemble = []
svc_ensemble = []
mlp_ensemble = []
decision_tree_ensemble = []
ensembles = [gaussian_ensemble,svc_ensemble,mlp_ensemble,decision_tree_ensemble]
names = ['gaussian','svc','mlp','decision tree']
for i in range(NUM_WEAK):
    gaussian_ensemble.append([GaussianNB(),[]])
    svc_ensemble.append([SVC(kernel='linear',probability =True),[]])
    mlp_ensemble.append([MLPClassifier(hidden_layer_sizes=(5, 2),max_iter=30,tol = 1e-1),[]])
    decision_tree_ensemble.append([DecisionTreeClassifier(max_depth=5,max_features=5),[]])
    



In [106]:
for i in range(len(ensembles)):
    acc = eval_single_weak(X,y,10,5,ensembles[i][0][0])
    print(f'First {names[i]:s} learners average Acc= {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

First gaussian learners average Acc= 0.91 ±0.084
First svc learners average Acc= 0.97 ±0.070
First mlp learners average Acc= 0.86 ±0.202
First decision tree learners average Acc= 0.87 ±0.129


In [107]:
import random
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .1,stratify=y)
def ensemble_fit(X_train,y_train,ensemble):
    
    for i in range(len(ensemble)):#ensuring each weak classifier gets at least 1 feature
        ensemble[i][1].append(i)
    for i in range(len(ensemble),len(X_train[0])):#the remaining features are given randomly to classifiers
        random.choice(ensemble)[1].append(i)
    for i in ensemble:
        subset = pd.DataFrame(X_train).iloc[:,[k for k in i[1]]]
        i[0].fit(subset,y_train)

for e in ensembles:
    ensemble_fit(X_train,y_train,e)


In [108]:
def ensemble_predict(X_test,y_test,ensemble):
    predictions = []
    for i in ensemble:
        subset = pd.DataFrame(X_test).iloc[:,[k for k in i[1]]]
        predictions.append(i[0].predict(subset))
        
    vote = []
    #print(len(predictions))
    for i in range(len(predictions[0])):
        count_0 = 0
        count_1 = 0
        for j in range(len(predictions)):#sorry for not using np.argmax
            if predictions[j][i]==0:
                count_0+=1
            else:
                count_1+=1
        if count_0>=count_1:#0 is poisonous, so >=
            vote.append(0)
        else:
            vote.append(1)
    return vote


In [109]:
for i in range(len(ensembles)):
    y_pred = ensemble_predict(X_test,y_test,ensembles[i])
    acc = [accuracy_score(y_test, y_pred)]
    print(f'Ensemble {names[i]:s} average Acc= {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Ensemble gaussian average Acc= 0.88 ±0.000
Ensemble svc average Acc= 0.99 ±0.000
Ensemble mlp average Acc= 0.93 ±0.000
Ensemble decision tree average Acc= 0.94 ±0.000
